- TensorFlow 模型文件是怎么样的？
- 如何保存一个 TensorFlow 模型？
- 如何恢复一个 TensorFlow 模型？
- 如何使用一个训练好的模型进行修改和微调？

##  TensorFlow 模型文件

在你训练完一个神经网络之后，你可能需要将这个模型保存下来，在后续实验中使用或者进行生产部署。那么，TensorFlow 模型文件长什么样呢？

TensorFlow 模型主要包含我们已经训练好的网络设计（计算图）和网络参数。因此，TensorFlow 模型主要有两个文件：

    元图（meta graph） 
    这是一个保存完整的 TensorFlow 图的协议缓存区，即所有的变量，操作，集合等等。该文件具有 .meta 扩展名。

    检查点文件（checkpoint file） 
    这是一个二进制文件，其中包含所有的权重，偏差，梯度和其他所有保存的值。这个文件的扩展名是 .ckpt。然而，TensorFlow 从版本 0.11 开始已经不再使用一个 .ckpt 文件来表示了，而是采用两个文件，如下：
        my_model.data-00000-of-00001
        my_model.index

.data 文件是包含我们训练变量的文件。除此之外，TensorFlow 还有一个名为 checkpoint 的文件，它用来保存最新检查点的记录。

所以，总而言之，TensorFlow 模型文件如下所示：

    my_model-1000.index
    my_model-1000.meta
    my_model-1000.data-00000-of-00001
    checkpoin

## 保存一个 TensorFlow 模型

### 概念

假设，你正在使用卷积神经网络训练一个图像分类模型。

一个传统做法是，你一直在关注损失值和正确率这两个指标，一旦你发现网络收敛了，那么你就会手动停止训练，或者你会运行固定的训练次数然后结束训练。

在完成训练之后，我们希望将所有的变量和计算图都保存起来，以便于未来使用。

所以，在 TensorFlow 中，我们可以使用 tf.train.Saver() 实例来保存所有的参数和计算图。

**saver = tf.train.Saver()**

请记住，TensorFlow 变量只在会话（session）中存在。因此，你必须在会话中，调用刚刚创建好的 save 方法，将模型保存起来。

**saver.save(sess, 'my-model')**

在这里， sess 是一个会话对象，而 my-test-model 就是你给模型起的名字。接下来，让我们看一个完整的例子：

### 示例

首先要建立一个 tf.train.Saver() 对象，用于保存/提取数据

再创建一个名为model的文件夹, 

最后用这个 saver 来保存变量到这个目录 "model/model.ckpt".

In [ ]:
!mkdir model

In [7]:
import tensorflow as tf 

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
# 可以指定saver保存什么变量，如果只保存v1，那么下边要恢复result就会出错，因为缺少v2
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "model/my_model.ckpt")
    #writer = tf.summary.FileWriter('model/SavedGraph',graph=tf.get_default_graph())
    #writer.close()

如果你想在 1000 次迭代之后再保存模型，那么我们可以通过传递步数来调用 save ，如下：

    saver.save(sess, 'my_model', global_step=1000)

这个代码会在模型文件之后添加上 -1000 ，如下：

    my_model-1000.index
    my_model-1000.meta
    my_model-1000.data-00000-of-00001
    checkpoint

[TensorFlow 模型文件](https://www.cnblogs.com/qwangxiao/p/9036493.html)

In [ ]:
# tensorboard --logdir=Saved_Graph

## 导入一个预训练的模型

如果你想使用别人的预训练好的模型进行微调，那么你需要做两件事：

    构建网络结构 
    你可以通过编写 Python 代码来重新构建整个网络，手动来创建网络中的每一个层。但是，因为我们已经将网络保存在 .meta 文件中了，我们可以直接使用 tf.train.import() 函数来导入这个模型，如下所示：saver = tf.train.import_meta_graph('my_model-1000.meta') 。

请记住，import_meta_graph 将 .meta 文件中定义的计算图数据直接附加到我们定义的当前图中。因此，这个操作会帮助我们创建计算图，但是我们仍然需要去加载计算图上面所有已经训练好的权重参数。

    加载参数 
    我们可以通过调用这个 tf.train.restore() 来恢复我们网络的参数。

In [10]:
with tf.Session() as sess:
    saver.restore(sess, "model/my_model.ckpt")
    print(result.eval())

INFO:tensorflow:Restoring parameters from model/my_model.ckpt
[3.]


如果不希望重复定义的模型，可以直接加载持久化的图：tf.train.import_meta_graph。

In [16]:
saver = tf.train.import_meta_graph("model/my_model.ckpt.meta")
with tf.Session() as sess:
    #saver.restore(sess, "model/my_model.ckpt")
    saver.restore(sess,tf.train.latest_checkpoint('model/'))
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))
    

INFO:tensorflow:Restoring parameters from model/my_model.ckpt
[3.]


## 变量重命名。

In [14]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})